In [3]:
import pandas as pd

# Load the datasets
df_hr = pd.read_csv('hr_terminations.csv')
df_app = pd.read_csv('app_users.csv')

# Convert date columns to actual datetime objects immediately
df_hr['Term_Date'] = pd.to_datetime(df_hr['Term_Date'])
df_app['Last_Login'] = pd.to_datetime(df_app['Last_Login'])

print(f"HR Records: {len(df_hr)}")
print(f"App Records: {len(df_app)}")
df_hr.head()

HR Records: 5
App Records: 6


,Employee_ID,Name,Term_Date,Department
0,E001,Alice Smith,2023-11-15,Sales
1,E005,Bob Johnson,2023-12-01,IT
2,E010,Charlie Brown,2024-01-10,Finance
3,E012,David Miller,2023-10-20,Marketing
4,E015,Eve Wilson,2023-12-25,Engineering


In [4]:
# Strip whitespace from IDs and Names to prevent 'false negatives'
df_hr['Employee_ID'] = df_hr['Employee_ID'].str.strip()
df_app['User_ID'] = df_app['User_ID'].str.strip()

# Standardizing names for easier visual review later
df_hr['Name'] = df_hr['Name'].str.strip().str.title()
df_app['Full_Name'] = df_app['Full_Name'].str.strip().str.title()

print("Data cleaning complete.")

Data cleaning complete.


In [5]:
# We join on the ID. 
# We use 'left' because we only care about people on the termination list.
audit_merge = pd.merge(
    df_hr, 
    df_app, 
    left_on='Employee_ID', 
    right_on='User_ID', 
    how='left'
)

# Display the merged table
audit_merge

,Employee_ID,Name,Term_Date,Department,User_ID,Full_Name,Account_Status,Last_Login
0,E001,Alice Smith,2023-11-15,Sales,E001,Alice Smith,Active,2024-01-05
1,E005,Bob Johnson,2023-12-01,IT,E005,Bob Johnson,Active,2023-11-28
2,E010,Charlie Brown,2024-01-10,Finance,E010,Charlie Brown,Disabled,2024-01-08
3,E012,David Miller,2023-10-20,Marketing,NaN,NaN,NaN,NaT
4,E015,Eve Wilson,2023-12-25,Engineering,E015,Eve Wilson,Active,2024-02-01


In [6]:
# 1. Identify Terminated but still 'Active' in Application
active_leavers = audit_merge[audit_merge['Account_Status'] == 'Active'].copy()

# 2. Identify Logins occurring AFTER termination date
# This is a critical security finding indicating potential account misuse
post_term_logins = audit_merge[audit_merge['Last_Login'] > audit_merge['Term_Date']].copy()

print(f"Finding 1: {len(active_leavers)} users still marked as 'Active'")
print(f"Finding 2: {len(post_term_logins)} users logged in after termination")

Finding 1: 3 users still marked as 'Active'
Finding 2: 2 users logged in after termination


In [8]:
%pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]━━ 1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Create a summary report
with pd.ExcelWriter('Termination_Audit_Report.xlsx') as writer:
    active_leavers.to_excel(writer, sheet_name='Active_Leavers', index=False)
    post_term_logins.to_excel(writer, sheet_name='Post_Term_Logins', index=False)
    audit_merge.to_excel(writer, sheet_name='Full_Traceability_Matrix', index=False)

print("Audit Report Exported: Termination_Audit_Report.xlsx")

Audit Report Exported: Termination_Audit_Report.xlsx
